- Project Id: ga-capstone-341005
- Project Number: 686873818286
- Project bucket: kendallfrimodig-earth-engine
- Bucket Folders
  - NAIP_infared
  - NAIP_natural
  - Polygon


In [ ]:
# Install GDAL and Geopandas
!apt install gdal-bin python-gdal python3-gdal --quiet
!apt install python3-rtree --quiet
!pip install git+git://github.com/geopandas/geopandas.git --quiet
!pip install descartes --quiet

# Folium 
!pip install folium

# Rasterio
!pip install Rasterio

# Install Pysal - Spatial Statistics
!pip install pysal --quiet
!pip install splot --quiet

Reading package lists...
Building dependency tree...
Reading state information...
gdal-bin is already the newest version (2.2.3+dfsg-2).
python-gdal is already the newest version (2.2.3+dfsg-2).
The following package was automatically installed and is no longer required:
  libnvidia-common-470
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  python3-numpy
Suggested packages:
  python-numpy-doc python3-nose python3-numpy-dbg
The following NEW packages will be installed:
  python3-gdal python3-numpy
0 upgraded, 2 newly installed, 0 to remove and 39 not upgraded.
Need to get 2,288 kB of archives.
After this operation, 13.2 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 python3-numpy amd64 1:1.13.3-2ubuntu1 [1,943 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 python3-gdal amd64 2.2.3+dfsg-2 [346 kB]
Fetched 2,288 kB in 2s (952 kB/s)
Selecting previously unselected package python3

# Housekeeping



## Libraries

In [ ]:
#pip install ipyleaflet GeoJSON
# import Earth Engine API and Cloud Storage API
import ee
from google.colab import auth
from googleapiclient.discovery import build
from apiclient.http import MediaIoBaseDownload
from googleapiclient.http import MediaFileUpload

# Data and Viz Libraries
import pandas as pd
import requests
#from IPython.display import GeoJSON


import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt

from osgeo import gdal
import folium
import rasterio as rio

import pysal
from pysal.lib  import weights
from pysal.explore import  esda
from esda.moran import Moran, Moran_Local
import splot
from splot.esda import moran_scatterplot, plot_moran, lisa_cluster

import urllib
import io
import numpy as np

import time

/usr/local/lib/python3.7/dist-packages/geopandas/_compat.py:115: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string
/usr/local/lib/python3.7/dist-packages/numba/np/ufunc/parallel.py:363: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB threading layer is disabled.
  warnings.warn(problem)
/usr/local/lib/python3.7/dist-packages/spaghetti/network.py:36: FutureWarning: The next major release of pysal/spaghetti (2.0.0) will drop support for all ``libpysal.cg`` geometries. This change is a first step in refactoring ``spaghetti`` that is expected to result in dramatically reduced runtimes for network instantiation and operations. Users currently requiring network and point pattern input as ``libpysal.cg`` ge

In [ ]:
# Trigger the authentication flow (Earth Engine).
ee.Authenticate()

# Initialize Earth Engine.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=MHC7i3T3lA6HXe1v6PQsUqJkYjCUb1cgSByV5FPGqsk&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWgLB38fFyhgY5gVf8dubXdi0KKsZl-PbS-tqkBDz9dHDCVDy-5a3Ts

Successfully saved authorization token.


In [ ]:
# Global Cloud Variables and Authenticate GCS
#auth.authenticate_user()
project_id = 'ga-capstone-341005'
bucketname = 'kendallfrimodig-earth-engine'
# Initialize Google Cloud Storage
#gcs_service = build('storage', 'v1')

In [ ]:
PROJECT = 'ga-capstone-341005'
!gcloud auth login --project {PROJECT}

Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=IaYVMEvYaOqeOX34DknrW3ZewAGyr0&prompt=consent&access_type=offline&code_challenge=-7v3NWASkog2uBCEuWzQnV7sF_xgpBbBVb2OB0UlpzI&code_challenge_method=S256

Enter verification code: 4/1AX4XfWiYGkNURexYSPqWQyq-XhESIVrBOldwUzyiUSPBGXZa9a9uSL33nPc

You are now logged in as [frimodke@mail.gvsu.edu].
Your current project is [ga-capstone-341005].  You can change this setting by running:
  $ gcloud config set project PROJECT_ID


## Define Functions

In [ ]:
KEY = 'mykey.json'
SERVICE_ACCOUNT='ga-capstone-341005@appspot.gserviceaccount.com'

!gcloud iam service-accounts keys create {KEY} --iam-account {SERVICE_ACCOUNT}

created key [414956a09b96dd4bf206bf107e454250cb1d34b8] of type [json] as [mykey.json] for [ga-capstone-341005@appspot.gserviceaccount.com]


In [ ]:
# Function for importing files from GCS
def import_gcs(filename):
  with open('/tmp/downloaded_from_gcs.txt', 'wb') as f:
    request = gcs_service.objects().get_media(bucket=bucketname,
                                              object=filename)
    media = MediaIoBaseDownload(f, request)
    done = False
    while not done:
      # _ is a placeholder for a progress object that we ignore.
      # (Our file is small, so we skip reporting progress.)
      _, done = media.next_chunk()
  print('Download complete')
  return media


# Function for exporting files to GCS
def export_gcs(filename, subdir, **kwargs):
  media = MediaFileUpload('/tmp/to_upload.txt', kwargs)

  request = gcs_service.objects().insert(bucket=(bucketname+subdir), 
                                        name=filename,
                                        media_body=media)

  response = None
  while response is None:
    # _ is a placeholder for a progress object that we ignore.
    # (Our file is small, so we skip reporting progress.)
    _, response = request.next_chunk()
  print('Upload complete')
  return 

A test to determine if earth engine is configured correctly. Output should be dictionary

In [ ]:
from google.auth.transport.requests import AuthorizedSession
from google.oauth2 import service_account

credentials = service_account.Credentials.from_service_account_file(KEY)
scoped_credentials = credentials.with_scopes(
    ['https://www.googleapis.com/auth/cloud-platform'])

session = AuthorizedSession(scoped_credentials)

url = 'https://earthengine.googleapis.com/v1alpha/projects/earthengine-public/assets/LANDSAT'

response = session.get(url)

from pprint import pprint
import json
pprint(json.loads(response.content))

{'id': 'LANDSAT',
 'name': 'projects/earthengine-public/assets/LANDSAT',
 'type': 'FOLDER'}


In [ ]:
from google.colab import drive
drive.mount(‘/content/gdrive’)

 Create list of image tiles in area of interest (coordinates of a rectangle drawn in earth engine)

In [ ]:
# bounding box for county

coords =  ([[-106.48758981209929, 39.21331227125566],
          [-106.48758981209929, 38.45917692072267],
          [-105.83802316170866, 38.45917692072267],
          [-105.83802316170866, 39.21331227125566]])


project = 'projects/earthengine-public'
asset_id = 'USDA/NAIP/DOQQ'
name = '{}/assets/{}'.format(project, asset_id)
url = 'https://earthengine.googleapis.com/v1alpha/{}:listImages?{}'.format(
  name, urllib.parse.urlencode({
    'startTime': '2019-01-01T00:00:00.000Z',
    'endTime': '2020-12-31T00:00:00.000Z',
    'region': '{"type":"LineString", "coordinates":' + str(coords) + '}',
}))

response = session.get(url)
content = response.content

tile_list = []

metadata = {}

for asset in json.loads(content)['images']:
    id = asset['id']
    tile_list.append(id)
    metadata[id] = asset['bands']


Parameters, tile name for each image in the area of interest.

In [ ]:
key =  tile_list[0]
print(key)
metadata[key][0]['grid']

USDA/NAIP/DOQQ/m_3810502_nw_13_060_20190922


{'affineTransform': {'scaleX': 0.6,
  'scaleY': -0.6,
  'translateX': 423966,
  'translateY': 4317342},
 'crsCode': 'EPSG:26913',
 'dimensions': {'height': 12310, 'width': 9800}}

In [ ]:
#print(tile_list)

# create if else, if each in(gcs/output/files without .tif) pass else execute

tile_list[7:]

In [ ]:
aoi = ee.FeatureCollection('projects/ga-capstone-341005/assets/AOI_chaffee')
def clip_export(img):
  tile = ee.Image(img).select(['R', 'B', 'G', 'N']);

  projection = tile.select('R').projection().getInfo();

  masked = tile.clip(aoi)


  task = ee.batch.Export.image.toCloudStorage(**{
    'image': masked,
    'description': "naip collection for chaffee county, CO",
    'bucket': 'kendallfrimodig-earth-engine',
    'fileNamePrefix': img,
    'skipEmptyTiles': True,
    'crs': projection['crs'],
    'crsTransform': projection['transform'],
    'fileFormat': 'GeoTIFF',
    'formatOptions': {'cloudOptimized': True},
    'maxPixels': 1e11
  })
  task.start()

  while task.active():
    print('Polling for task (id: {}).'.format(task.id))
    time.sleep(5)

  return

In [ ]:
for each in tile_list[25:]:
  clip_export(each)

  

In [ ]:
len(tile_list)

35

Method of Visualizing Earth Engine Objects in Python Client

            # Import the Folium library.
            import folium

            # Define a method for displaying Earth Engine image tiles to folium map.
            def add_ee_layer(self, ee_image_object, vis_params, name):
              map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
              folium.raster_layers.TileLayer(
                tiles = map_id_dict['tile_fetcher'].url_format,
                attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
                name = name,
                overlay = True,
                control = True
              ).add_to(self)

            # Add EE drawing method to folium.
            folium.Map.add_ee_layer = add_ee_layer

            # Set visualization parameters.
            #vis_params = {
             # 'min': 0,
             # 'max': 4000,
             # 'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']}

            # Create a folium map object.
            my_map = folium.Map(location=[20, 0], zoom_start=3)

            # Add the elevation model to the map object.
            #my_map.add_ee_layer(dem.updateMask(dem.gt(0)), vis_params, 'DEM')


            my_map.add_ee_layer(NAIP_n, vis_params, 'NAIP_natural')
            # Add a layer control panel to the map.
            my_map.add_child(folium.LayerControl())

            # Display the map.
            display(my_map)

Takes images and outputs numpy array. Maxed out at 10,000 x 10,000 pixels for public earth engine assets, will try on assets from GCS


        key =  tile_list[0]
        asset_id = key
        name = '{}/assets/{}'.format(project, asset_id)
        url = 'https://earthengine.googleapis.com/v1alpha/{}:getPixels'.format(name)
        body = json.dumps({
            'fileFormat': 'NPY',
            'bandIds': ['R', 'B', 'G', 'N'],
            'grid': metadata[key][0]['grid']
        })

        pixels_response = session.post(url, body)
        pixels_content = pixels_response.content

        array = np.load(io.BytesIO(pixels_content), allow_pickle=True)
        print('Shape: %s' % (array.shape,))
        print('Data:')
        print(array)

